In [7]:
import os
import pickle
import numpy as np
import pandas as pd
import string
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import spacy

# from nltk import word_tokenize, sent_tokenize
from string import punctuation
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [8]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('omw-1.4')

In [9]:
spacy.prefer_gpu()

True

In [10]:
nlp = spacy.load("en_core_web_sm")

# Preprocessing

## Functions

### Sunday 12 March

TO DO FOR the WEEK: <br>
    - add function to remove stopwords from pos_id X
1. og statement -> get pos -> pos_id
	- train_data['pos_id'] = train_data['statement'].apply(get_pos)

2. og statement -> # preprocess statement: words lemmatized and stopwords are removed -> pred_statement
	- train_data['pred_statement'] = train_data['statement'].apply(preprocess_statement)

3. pred_statement -> add to vocab dict: add preprocessed statements to vocab dict -> vocab_dict
	- vocabulary_dict = load_statement_vocab_dict(train_data)

4. vocab_dict, pred_statement -> getWordID(): turn preprocessed statements to vector -> val
	- train_data['word_id'] = train_data['pred_statement'].apply(lambda x: getWordId(x, vocabulary_dict)

**NOTE** it is generally fine to add pos tags without stopwords removed.
	in siddarthhari, padding used to put pos tags as separate input layer.

15 mar: <br>
    - extracted getWordId() from preprocess_statement()
    


### 19 March
TODO: 
1. POS tags - incorporate stop words into pos tags 
- update: no need since separate input.
2. preprocess_statement() - configure best ones. confirm lemmatizer works

In [32]:
pos_tags = { 'ADJ': 'adjective', 'ADP': 'adposition', 'ADV': 'adverb', 'AUX': 'auxiliary verb', 'CONJ': 'coordinating conjunction',
                    'DET': 'determiner', 'INTJ': 'interjection', 'NOUN': 'noun',  'NUM': 'numeral', 'PART': 'particle', 'PRON': 'pronoun',
                    'PROPN': 'proper noun', 'PUNCT': 'punctuation', 'X': 'other', 'SCONJ': 'subordinating conjunction', 'SYM': 'symbol', 'VERB': 'verb'}

# proposed pos_dict: values correspond to pos tag importance
dict1 = {'NOUN': 0, 'VERB': 1, 'ADJ': 2, 'ADV': 3, 'ADP': 4, 'PRON': 5,
              'NUM': 6, 'SCONJ': 7, 'PROPN': 8, 'CONJ': 9, 'PUNCT': 10, 'AUX': 11,
              'PART': 12, 'INTJ': 13, 'DET': 14, 'SYM': 14, 'X': 14}

dict2 = {'ADJ': 0, 'ADP': 1, 'ADV': 2, 'AUX': 3, 'CONJ': 4,
              'DET': 5, 'INTJ': 6, 'NOUN': 7, 'NUM': 8, 'PART': 9,
              'PRON': 10, 'PROPN': 11, 'PUNCT': 12, 'X': 13,  'SCONJ': 14, 'SYM': 15, 'VERB': 16}

In [12]:
def get_pos(statement, pos_dict):
    doc = nlp(statement)
    taglist = []
    for token in doc:
        # labels stopwords from list of pos tags as 12
        # if token.is_stop:
        #     taglist.append(12)
        # else:
            taglist.append(pos_dict.get(token.pos_, max(pos_dict.values())))
    return taglist

In [13]:
def preprocess_statement2(statement, pos_remove=None, spacy_stopword_removal=False):
# lemmatizes and lowercases tokens that are not punctuation or whitespace.
# params: statement - string
#         exclude_pos - list of POS tags to exclude from word list
#         remove_stopwords - removes stopwords based on pretrained spacy model
#         **warning** not recommended to toggle both params to True
# returns: tokens joined into a string
    doc = nlp(statement)
    words = []
    for token in doc:
        # check if token should be excluded based on part of speech or stopword status
        if pos_remove and token.pos_ in pos_remove:
            continue
        if spacy_stopword_removal and token.is_stop:
            continue
        # only include tokens that are not punctuation or whitespace
        if not token.is_punct and not token.is_space:
            # lemmatize and lowercase
            words.append(token.lemma_.lower())
    return ' '.join(words)

In [14]:
# checks if vocabulary.p exists, if not creates it
def load_statement_vocab_dict(train_data):
    if not os.path.exists('vocabulary.p'):
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(train_data['pred_statement'])
        vocabulary_dict = tokenizer.word_index
        print(len(vocabulary_dict))
        with open("vocabulary.p", "wb") as f:
            pickle.dump(vocabulary_dict, f)
        print('Created Vocabulary Dictionary...')
        print('Saved Vocabulary Dictionary...')
    else:
        print('Loading Vocabulary Dictionary...')
        with open("vocabulary.p", "rb") as f:
            vocabulary_dict = pickle.load(f)
    return vocabulary_dict

In [15]:
def create_statement_vocab_dict2(data, column_name):

    vocab_file_name = f"vocabulary_{column_name}.p"

    if os.path.exists(vocab_file_name):
        with open(vocab_file_name, "rb") as f:
            old_vocabulary_dict = pickle.load(f)
        if column_name in old_vocabulary_dict:
            os.remove(vocab_file_name)
            print(f"Deleted existing Vocabulary Dictionary for column {column_name}...")
        else:
            print(f"Existing Vocabulary Dictionary does not match column {column_name}. Creating new Vocabulary Dictionary...")

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data[column_name])
    vocabulary_dict = tokenizer.word_index
    print(f"Created Vocabulary Dictionary for column {column_name}.")
    print('Size: ' + str(len(vocabulary_dict)))
    with open(vocab_file_name, "wb") as f:
        pickle.dump(vocabulary_dict, f)
    print(f"Saved Vocabulary Dictionary as {vocab_file_name}")

    return vocabulary_dict

In [16]:
def create_statement_vocab_dict3(data, column_name):
    vocab_file_name = f"vocabulary_{column_name}.p"

    if os.path.exists(vocab_file_name):
        print(f"Loading existing Vocabulary Dictionary for column {column_name}...")
        with open(vocab_file_name, "rb") as f:
            vocabulary_dict = pickle.load(f)
        print(f'Size: {len(vocabulary_dict)}')
    else:
        print(f"Creating new Vocabulary Dictionary for column {column_name}...")
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(data[column_name])
        vocabulary_dict = tokenizer.word_index
        print(f"Created Vocabulary Dictionary for column {column_name}.")
        print(f'Size: {len(vocabulary_dict)}')
        with open(vocab_file_name, "wb") as f:
            pickle.dump(vocabulary_dict, f)
        print(f"Saved Vocabulary Dictionary as {vocab_file_name}")

    return vocabulary_dict

In [17]:
def create_statement_vocab_dict(data, column_name):
    vocab_file_name = f"vocabulary_{column_name}.p"

    if os.path.exists(vocab_file_name):
        with open(vocab_file_name, "rb") as f:
            old_vocabulary_dict = pickle.load(f)
        # Check length of existing vocabulary dictionary and length of vocabulary dictionary created from the text data
        if len(old_vocabulary_dict) == len(set(data[column_name].str.split().sum())):
            print(f"Using existing Vocabulary Dictionary for column {column_name}...")
            return old_vocabulary_dict

        # If the length is different, delete the existing vocabulary dictionary file
        else:
            os.remove(vocab_file_name)
            print(f"Deleted existing Vocabulary Dictionary for column {column_name}...")

    # Create a new tokenizer object and fit it on the text data for the specified column
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data[column_name])

    vocabulary_dict = tokenizer.word_index
    print(f"Created Vocabulary Dictionary for column {column_name}.")
    print('Size: ' + str(len(vocabulary_dict)))

    # Save the vocabulary dictionary to a file
    with open(vocab_file_name, "wb") as f:
        pickle.dump(vocabulary_dict, f)
    print(f"Saved Vocabulary Dictionary as {vocab_file_name}")
    return vocabulary_dict


In [18]:
# preprocessed statement to vector
def getWordId(pred_statement, vocabulary_dict):
    text = text_to_word_sequence(pred_statement)
    val = [vocabulary_dict[t] for t in text if t in vocabulary_dict]
    return val

In [19]:
# # preprocessed statement to vector
# def getWordId(pred_statement, vocab_file_name):
#     with open(vocab_file_name, "rb") as f:
#         vocabulary_dict = pickle.load(f)
#     text = text_to_word_sequence(pred_statement)
#     val = [vocabulary_dict[t] for t in text if t in vocabulary_dict]
#     return val

In [20]:
def print_row_info(row_num, df):
    print(f"#### label #####\n{df.loc[row_num, 'label']}")
    print(f"#### original statement #####\n{df.loc[row_num, 'statement']}")
    print(f"#### pos_id #####\n{df.loc[row_num, 'pos_id']}")
    print(f"#### custom stopwords #####\n{df.loc[row_num, 'pred_statement']}")
    print(f"#### word_id #####\n{df.loc[row_num, 'word_id_custom']}")
    print(f"#### spacy stopwords #####\n{df.loc[row_num, 'pred_statement_spacyswr']}")
    print(f"#### spacy word_id #####\n{df.loc[row_num, 'word_id_spacy']}")

In [21]:
# train_data.tail()

NameError: name 'train_data' is not defined

In [22]:
def clean_data(data, label_map):
    # maps labels to numerical values
    data['label'] = data['label'].map(label_map)

    # replace missing values in 'subject' and 'speaker' with empty strings
    data['subject'] = data['subject'].fillna('')
    data['speaker'] = data['speaker'].fillna('')

    # drop rows where there are missing values in 'statement'
    data.drop(index=data[data.statement==' '].index, inplace=True)
    data.drop(index=data[data.statement=='  '].index, inplace=True)
    data.drop(index=data[data.statement=='\n'].index, inplace=True)

    # drop context column
    data.drop(columns=['context'], inplace=True)
    data.drop(columns=['id'], inplace=True)
    data.drop(columns=['state info'], inplace=True)
    data.drop(columns=['party'], inplace=True)

    # reset index
    data = data.reset_index()

    return data

## DataFrame Cleaning

- train_data['label'] -> num values
- drops all rows where 'subject' and 'speaker' columns are missing
- drops all rows that have missing values in 'statement'
- drops context column

<h3>Mar 19 todo:</h3>

- refactor to single function
- instead of dropping rows, replace missing values in 'subject' and 'speaker' as empty string

In [23]:
# column titles and label maps
columns = ['id','label','statement','subject','speaker','job title','state info','party','barely true','false','half-true','mostly-true','pants-on-fire','context']
label_map = {'pants-fire':-3, 'false':-2, 'barely-true':-1, 'half-true':1, 'mostly-true':2, 'true':3}

In [24]:
# load dataset files as pandas dataframes
train_data = pd.read_csv('train.tsv', sep='\t', header=None, names=columns)
val_data = pd.read_csv('valid.tsv', sep='\t', header=None, names=columns)
test_data = pd.read_csv('test.tsv', sep='\t', header=None, names=columns)

# train_data.head()

In [25]:
train_data = clean_data(train_data, label_map)
val_data = clean_data(val_data, label_map)
test_data = clean_data(test_data, label_map)

In [27]:
train_data.head()

,index,label,statement,subject,speaker,job title,barely true,false,half-true,mostly-true,pants-on-fire
0,0,-2,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,0.0,1.0,0.0,0.0,0.0
1,1,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,0.0,0.0,1.0,1.0,0.0
2,2,2,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,70.0,71.0,160.0,163.0,9.0
3,3,-2,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,7.0,19.0,3.0,5.0,44.0
4,4,1,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,15.0,9.0,20.0,19.0,2.0


In [28]:
print('real news count')
train_data[train_data['label']>0].count()

real news count


index            5752
label            5752
statement        5752
subject          5752
speaker          5752
job title        4264
barely true      5752
false            5752
half-true        5752
mostly-true      5752
pants-on-fire    5752
dtype: int64

### for this experiment, siddarth's results will be used as benchmark.


#### to test the different preprocessing methods, the following experiments will be run:
1. siddarth - baseline
    - ogdict
    - nltk stopword removal only
2. fathan - pos tag check
    - dict1
    - spacy stopword removal only
    - glove
3. fathan - preprocess
    - dict1
    - full preprocessing
    - glove

### get pos tag

In [29]:
pos_tags = {
    'ADJ': 'adjective', 'ADP': 'adposition', 'ADV': 'adverb',
    'AUX': 'auxiliary verb', 'CONJ': 'coordinating conjunction',
    'DET': 'determiner', 'INTJ': 'interjection', 'NOUN': 'noun',
    'NUM': 'numeral', 'PART': 'particle', 'PRON': 'pronoun',
    'PROPN': 'proper noun', 'PUNCT': 'punctuation', 'X': 'other',
    'SCONJ': 'subordinating conjunction', 'SYM': 'symbol', 'VERB': 'verb'
    }

# proposed pos_dict: values correspond to pos tag importance
dict1 = {
    'NOUN': 0, 'VERB': 1, 'ADJ': 2, 'ADV': 3, 'ADP': 4, 'PRON': 5,
    'NUM': 6, 'SCONJ': 7, 'PROPN': 8, 'CONJ': 9, 'PUNCT': 10, 'AUX': 11,
    'PART': 12, 'INTJ': 13, 'DET': 14, 'SYM': 14, 'X': 14
    }

# unordered pos_dict
dict2 = {
    'ADJ': 0, 'ADP': 1, 'ADV': 2, 'AUX': 3, 'CONJ': 4,
    'DET': 5, 'INTJ': 6, 'NOUN': 7, 'NUM': 8, 'PART': 9,
    'PRON': 10, 'PROPN': 11, 'PUNCT': 12, 'X': 13,
    'SCONJ': 14, 'SYM': 15, 'VERB': 16
    }

In [30]:
train_data['pos_id'] = train_data['statement'].apply(get_pos, pos_dict=dict1)
train_data['pos2_id'] = train_data['statement'].apply(get_pos, pos_dict=dict2)
# val_data['pos_id'] = val_data['statement'].apply(get_pos, pos_dict=dict1)
# test_data['pos_id'] = test_data['statement'].apply(get_pos, pos_dict=dict1)

In [31]:
train_data.head()

,index,label,statement,subject,speaker,job title,barely true,false,half-true,mostly-true,pants-on-fire,pos_id,pos2_id
0,0,-2,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,0.0,1.0,0.0,0.0,0.0,"[1, 14, 8, 8, 2, 0, 1, 2, 10, 0, 0, 4, 0, 10]","[16, 5, 11, 11, 0, 7, 16, 0, 12, 7, 7, 1, 7, 12]"
1,1,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,0.0,0.0,1.0,1.0,0.0,"[7, 1, 14, 0, 4, 0, 0, 10, 5, 1, 7, 2, 0, 1, 4...","[14, 16, 5, 7, 1, 7, 7, 12, 10, 16, 14, 0, 7, ..."
2,2,2,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,70.0,71.0,160.0,163.0,9.0,"[8, 8, 1, 4, 8, 8, 10, 4, 1, 12, 1, 8, 8, 14, ...","[11, 11, 16, 1, 11, 11, 12, 1, 16, 9, 16, 11, ..."
3,3,-2,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,7.0,19.0,3.0,5.0,44.0,"[0, 0, 0, 0, 11, 2, 12, 1, 2, 0, 0, 0, 10]","[7, 7, 7, 7, 3, 0, 9, 16, 0, 7, 7, 7, 12]"
4,4,1,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,15.0,9.0,20.0,19.0,2.0,"[14, 2, 0, 1, 4, 14, 0, 4, 5, 0, 10]","[5, 0, 7, 16, 1, 5, 7, 1, 10, 7, 12]"


### preprocess statements, get word_id

In [33]:
pos_exclusion_list = ['SYM', 'DET', 'X', 'PRON', 'PART', 'CONJ', 'ADP']
# pos_exclusion_list2 = ['']
# preprocess_statement2(train_data['statement'], exclude_pos=prep_pos_dict, remove_stopwords=True)

In [34]:
# custom stopword removal
train_data['pred_statement'] = train_data['statement'].apply(preprocess_statement2, pos_remove=pos_exclusion_list, spacy_stopword_removal=False)
# spacy stopword removal
train_data['pred_statement_spacyswr'] = train_data['statement'].apply(preprocess_statement2, pos_remove=None, spacy_stopword_removal=True)

# val_data['pred_statement'] = val_data['statement'].apply(preprocess_statement)
# test_data['pred_statement'] = test_data['statement'].apply(preprocess_statement)
# train_data.head()

In [43]:
train_data.head()

,index,label,statement,subject,speaker,job title,barely true,false,half-true,mostly-true,pants-on-fire,pos_id,pos2_id,pred_statement,pred_statement_spacyswr,word_id_custom,word_id_spacy
0,0,-2,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,0.0,1.0,0.0,0.0,0.0,"[1, 14, 8, 8, 2, 0, 1, 2, 10, 0, 0, 4, 0, 10]","[16, 5, 11, 11, 0, 7, 16, 0, 12, 7, 7, 1, 7, 12]",say annies list political group support third ...,say annies list political group support trimes...,"[3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...","[1, 5315, 633, 423, 332, 37, 679, 3919, 120, 936]"
1,1,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,0.0,0.0,1.0,1.0,0.0,"[7, 1, 14, 0, 4, 0, 0, 10, 5, 1, 7, 2, 0, 1, 4...","[14, 16, 5, 7, 1, 7, 7, 12, 10, 16, 14, 0, 7, ...",when do decline coal start start when natural ...,decline coal start start natural gas take star...,"[37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...","[6135, 8949, 720, 773, 249, 249, 6135, 891, 20..."
2,2,2,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,70.0,71.0,160.0,163.0,9.0,"[8, 8, 1, 4, 8, 8, 10, 4, 1, 12, 1, 8, 8, 14, ...","[11, 11, 16, 1, 11, 11, 12, 1, 16, 9, 16, 11, ...",hillary clinton agree john mccain vote give ge...,hillary clinton agree john mccain vote george ...,"[104, 69, 734, 160, 201, 18, 89, 262, 137, 258...","[74, 49, 649, 125, 157, 12, 127, 212, 103, 208..."
3,3,-2,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,7.0,19.0,3.0,5.0,44.0,"[0, 0, 0, 0, 11, 2, 12, 1, 2, 0, 0, 0, 10]","[7, 7, 7, 7, 3, 0, 9, 16, 0, 7, 7, 7, 12]",health care reform legislation be likely manda...,health care reform legislation likely mandate ...,"[19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...","[13, 16, 165, 202, 514, 342, 301, 372, 140, 2747]"
4,4,1,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,15.0,9.0,20.0,19.0,2.0,"[14, 2, 0, 1, 4, 14, 0, 4, 5, 0, 10]","[5, 0, 7, 16, 1, 5, 7, 1, 10, 7, 12]",economic turnaround start end term,economic turnaround start end term,"[282, 3331, 308, 247, 248]","[224, 3208, 249, 198, 199]"


In [42]:
vocabulary_dict_custom = create_statement_vocab_dict(train_data, 'pred_statement')

Deleted existing Vocabulary Dictionary for column pred_statement...
Created Vocabulary Dictionary for column pred_statement.
Size: 9606
Saved Vocabulary Dictionary as vocabulary_pred_statement.p


In [37]:
vocabulary_dict_spacy = create_statement_vocab_dict(train_data, 'pred_statement_spacyswr')

Deleted existing Vocabulary Dictionary for column pred_statement_spacyswr...
Created Vocabulary Dictionary for column pred_statement_spacyswr.
Size: 9495
Saved Vocabulary Dictionary as vocabulary_pred_statement_spacyswr.p


In [ ]:
# vocab_file_name = "vocabulary_pred_statement.p"
# train_data['word_id'] = train_data['pred_statement'].apply(lambda x: getWordId(x, vocab_file_name))

In [38]:
train_data['word_id_custom'] = train_data['pred_statement'].apply(lambda x: getWordId(x, vocabulary_dict_custom))
train_data['word_id_spacy'] = train_data['pred_statement'].apply(lambda x: getWordId(x, vocabulary_dict_spacy))
# val_data['word_id'] = val_data['pred_statement'].apply(lambda x: getWordId(x, vocabulary_dict))
# test_data['word_id'] = test_data['pred_statement'].apply(lambda x: getWordId(x, vocabulary_dict))
# train_data.drop('pred_statement', axis=1, inplace=True)

In [43]:
train_data.head()

,level_0,index,label,statement,subject,speaker,job title,barely true,false,half-true,mostly-true,pants-on-fire,pred_statement,pred_statement_spacyswr,word_id_custom,word_id_spacy
0,0,0,NaN,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,0.0,1.0,0.0,0.0,0.0,say annies list political group support third ...,say annies list political group support trimes...,"[3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...","[1, 5315, 633, 423, 332, 37, 679, 3919, 120, 936]"
1,1,1,NaN,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,0.0,0.0,1.0,1.0,0.0,when do decline coal start start when natural ...,decline coal start start natural gas take star...,"[37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...","[6135, 8949, 720, 773, 249, 249, 6135, 891, 20..."
2,2,2,NaN,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,70.0,71.0,160.0,163.0,9.0,hillary clinton agree john mccain vote give ge...,hillary clinton agree john mccain vote george ...,"[104, 69, 734, 160, 201, 18, 89, 262, 137, 258...","[74, 49, 649, 125, 157, 12, 127, 212, 103, 208..."
3,3,3,NaN,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,7.0,19.0,3.0,5.0,44.0,health care reform legislation be likely manda...,health care reform legislation likely mandate ...,"[19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...","[13, 16, 165, 202, 514, 342, 301, 372, 140, 2747]"
4,4,4,NaN,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,15.0,9.0,20.0,19.0,2.0,economic turnaround start end term,economic turnaround start end term,"[282, 3331, 308, 247, 248]","[224, 3208, 249, 198, 199]"


### compare statement, pred_statement, word_id, pos_id

In [41]:
print_row_info(192, train_data)

#### label #####
-3
#### original statement #####
Says that in a hearing, Rep. Gabrielle Giffords suggested to Gen. David Petraeus that the Army put more emphasis on less environmentally damaging methods, like stabbing or clubbing enemy forces in order to minimize the carbon output.
#### custom stopwords #####
say that hearing rep. gabrielle giffords suggest gen. david petraeus that army put more emphasis less environmentally damaging method stab or club enemy force order minimize carbon output
#### word_id #####
[3, 10, 1266, 227, 5514, 5515, 922, 2562, 431, 2900, 10, 1147, 197, 8, 5516, 146, 4084, 3363, 2563, 4085, 31, 1351, 2288, 188, 469, 4086, 1203, 3364]
#### spacy stopwords #####
say hearing rep. gabrielle giffords suggest gen. david petraeus army emphasis environmentally damaging method like stab club enemy force order minimize carbon output
#### spacy word_id #####
[1, 1168, 180, 5389, 5390, 831, 2449, 364, 2780, 1053, 879, 5391, 9029, 3963, 3241, 2450, 3964, 1251, 2181, 150, 

In [34]:
print_row_info(61, train_data)

#### label #####
2
#### original statement #####
Said he's the only Republican candidate "who's actually turned around a government economy."
#### custom stopwords #####
say be only republican candidate be actually turn government economy
#### word_id #####
[3, 1, 40, 71, 149, 1, 186, 446, 44, 179]
#### spacy stopwords #####
say republican candidate actually turn government economy
#### spacy word_id #####
[1, 50, 113, 148, 379, 30, 142]


In [50]:
print_row_info(61, train_data)

#### label #####
2
#### original statement #####
Said he's the only Republican candidate "who's actually turned around a government economy."
#### pos_id #####
[1, 5, 11, 14, 2, 2, 0, 10, 5, 11, 3, 1, 4, 14, 0, 0, 10, 10]
#### preprocessed statement #####
say republican candidate actually turn government economy
#### word_id #####
[1, 49, 111, 145, 377, 29, 139]


In [27]:
# 22 mar 11:25 AM
print_row_info(61, train_data)

#### label #####
2
#### original statement #####
Said he's the only Republican candidate "who's actually turned around a government economy."
#### preprocessed statement #####
say he be only republican candidate who be actually turn around government economy
#### word_id #####
[6, 25, 1, 65, 100, 181, 52, 1, 227, 494, 580, 71, 220]


In [21]:
# 21 mar 1:20 pm
print_row_info(2000, train_data)

#### label #####
2
#### original statement #####
The DREAM Act was written by members of both parties. When it came up for a vote a year and a half ago, Republicans in Congress blocked it. The bill hadnt changed. ... The only thing that had changed was politics.
#### preprocessed statement #####
dream act be write by member of party when it come up for vote year and half ago republicans in congress block it bill have not change only thing that have change be politic
#### word_id #####
[1390, 209, 1, 551, 19, 243, 3, 257, 62, 20, 123, 78, 8, 35, 12, 7, 236, 327, 223, 2, 150, 753, 20, 51, 5, 11, 217, 65, 451, 9, 5, 217, 1, 1612]


In [ ]:
# 21 mar 10:00 am
print_row_info(100, train_data)

In [ ]:
# removed det, sym, etc from pred_statement
print_row_info(2000, train_data)

In [ ]:
# stop words = 17
print_row_info(100, train_data)

In [ ]:
# print(train_data['pred_statement']) 
ROW = 101
print(f"original statements:\n{train_data.loc[ROW, 'statement']}")
print(f"pos tag ids:\n{train_data.loc[ROW, 'pos_id']}") 
print(f"preprocessed statements:\n{train_data.loc[ROW, 'pred_statement']}")
print(f"pred_statements vectorized:\n{train_data.loc[ROW, 'word_id']}")
print(train_data.loc[ROW, 'label'])

In [ ]:
# # if not token.is_punct and not token.is_space
# ROW = 100
# print(f"original statements:\n{train_data.loc[ROW, 'statement']}")
# print(f"pos tag ids:\n{train_data.loc[ROW, 'pos_id']}")
# print(f"preprocessed statements:\n{train_data.loc[ROW, 'pred_statement']}")
# print(f"pred_statements vectorized:\n{train_data.loc[ROW, 'word_id']}")
# print(train_data.loc[ROW, 'label'])

In [ ]:
# # if not token.is_punct and not token.is_space and token.pos_ not in ['DET', 'ADP', 'CONJ', 'PRON']: # and not token.is_stop:
# ROW = 4
# print(f"original statements:\n{train_data.loc[ROW, 'statement']}")
# print(f"pos tag ids:\n{train_data.loc[ROW, 'pos_id']}")
# print(f"preprocessed statements:\n{train_data.loc[ROW, 'pred_statement']}")
# print(f"pred_statements vectorized:\n{train_data.loc[ROW, 'word_id']}")
# print(train_data.loc[ROW, 'label'])

# Word Embeddings

In [45]:
print_row_info(192, train_data)

#### label #####
-3
#### original statement #####
Says that in a hearing, Rep. Gabrielle Giffords suggested to Gen. David Petraeus that the Army put more emphasis on less environmentally damaging methods, like stabbing or clubbing enemy forces in order to minimize the carbon output.
#### custom stopwords #####
say that hearing rep. gabrielle giffords suggest gen. david petraeus that army put more emphasis less environmentally damaging method stab or club enemy force order minimize carbon output
#### word_id #####
[3, 10, 1266, 227, 5514, 5515, 922, 2562, 431, 2900, 10, 1147, 197, 8, 5516, 146, 4084, 3363, 2563, 4085, 31, 1351, 2288, 188, 469, 4086, 1203, 3364]
#### spacy stopwords #####
say hearing rep. gabrielle giffords suggest gen. david petraeus army emphasis environmentally damaging method like stab club enemy force order minimize carbon output
#### spacy word_id #####
[1, 1168, 180, 5389, 5390, 831, 2449, 364, 2780, 1053, 879, 5391, 9029, 3963, 3241, 2450, 3964, 1251, 2181, 150, 

In [46]:
# define function to read GloVe embeddings
def read_glove_embeddings():
    embeddings = {}
    with open("glove.6B.100d.txt", encoding="utf8") as file_object:
        for line in file_object:
            word_embed = line.split()
            word = word_embed[0]
            embed = np.array(word_embed[1:], dtype="float32")
            # if word.lower() in dictionary:
            embeddings[word.lower()] = embed
    return embeddings

# # create embedding matrix for vocabulary_dict1
# num_words1 = len(vocabulary_dict1) + 1
# embedding_matrix1 = np.zeros((num_words1, EMBED_DIM))
#
# for word, i in vocabulary_dict1.items():
#     embedding_vector = embeddings_dict1.get(word)
#     if embedding_vector is not None:
#         embedding_matrix1[i] = embedding_vector
#
# # create embedding matrix for vocabulary_dict2
# num_words2 = len(vocabulary_dict2) + 1
# embedding_matrix2 = np.zeros((num_words2, EMBED_DIM))
#
# for word, i in vocabulary_dict2.items():
#     embedding_vector = embeddings_dict2.get(word)
#     if embedding_vector is not None:
#         embedding_matrix2[i] = embedding_vector


In [47]:
# read GloVe embeddings
embeddings_dict = read_glove_embeddings()
print(len(embeddings_dict), " : Word Embeddings Found")

400001  : Word Embeddings Found


In [48]:
def create_embedding_matrix(vocabulary_dict, embeddings_dictionary, embed_dim):
    num_words = len(vocabulary_dict) + 1
    embedding_matrix = np.zeros((num_words, embed_dim))

    for word, i in vocabulary_dict.items():
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [49]:
EMBED_DIM = 100
embedding_matrix_custom = create_embedding_matrix(vocabulary_dict_custom, embeddings_dict, EMBED_DIM)
embedding_matrix_spacy = create_embedding_matrix(vocabulary_dict_spacy, embeddings_dict, EMBED_DIM)

In [ ]:
# reads pre-trained GloVe embeddings and store in a dictionary
# embeddings = {}
# with open("glove.6B.100d.txt", encoding="utf8") as file_object:
#     for line in file_object:
#         word_embed = line.split()
#         word = word_embed[0]
#         embed = np.array(word_embed[1:], dtype="float32")
#         embeddings[word.lower()] = embed
#
# print(len(embeddings), " : Word Embeddings Found")
# print(len(embeddings[word]), " : Embedding Dimension")
#
# EMBED_DIM = 100
# num_words = len(vocabulary_dict) + 1
# embedding_matrix = np.zeros((num_words, EMBED_DIM))
#
# # assign embedding vector to each word in vocabulary_dict
# for word, i in vocabulary_dict.items():
#     embedding_vector = embeddings.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector
#
# embeddings_index = None